<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Open Source Decision Intelligence

Decision Intelligence applied in this module:  
* Listing key factors to consider when making a sound decision  
* Decision Scenario: Use a decision framework (Ben Franklin's Pro & Con List) to create a decision plan  
* Improving Decision Intelligence process by explicitly proving decision frameworks and additional context 

A recommended enterprise pattern is to scale Articial Intelligence strategy with three key areas:
* Commercial AI (OpenAI and other proprietary AI providers)
* Open-Source AI (open-source AI providers)
* Partner (Vendor) AI (i.e. company HR Software, contract software)  

These three areas together strategically form AI capability and capacity in what I like to call the "AI Brain". This is illustrated below.

<img width ="750px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/AIBrainPillars.png">

Semantic Kernel embraces AI orchestration across all the pillars mentioned above. It allows all types of models (commercial or proprietary) and almost any APIs to be orchestrated to faciliate better decision intelligence.

### Step 1 - Get Started with LMStudio and Local Open Source AI Models 

This module highlights how to use local GenAI models (i.e. LLMs) with Semantic Kernel. To illustrate this the Phi-3.5 model is run locally using LMStudio as a local REST endpoint that will interface with Semantic Kernel orchestration.  

Steps to get started:
* Download & install LMStudio: https://lmstudio.ai/ (Windows, Mac or Linux) 
* Run the LMStudio studio application 
* In the LMStudio application, download the Phi-3.5-Mini GGUF file. LMStudio will inspect your hardware and let you know which quantized version of the model(s) is optimal for your hardware. Note: Even computers with small graphics cards can run these models well locally. Furthermore, laptops such as the Macbook Pro with Neural Engine can run LMStudio local models as well. 
* Start the LMStudio Server with the Phi-3.5-Mini model loaded. This will start a local REST endpoint with a URI similar to http://localhost:1234/v1 
* The LMStudio local server does not have default security, you can simply check by navigating to this link in any browser to check if a model is loaded: http://localhost:1234/v1/models 

<img width ="900px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/LMStudioServer.png">

### Step 2 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next cell to:
* Use the Configuration Builder to use the local LMStudio Server  
* Use the API configuration to build the Semantic Kernel orchestrator  
* Notice there is no security being passed in and it is simply a URL

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.29.0"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

#pragma warning disable SKEXP0010 
var semanticKernel = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion(
        modelId: "lmstudio-community/Phi-3.5-mini-instruct-GGUF",
        endpoint: new Uri("http://localhost:1234/v1/"),
        apiKey: null)
    .Build();
#pragma warning restore SKEXP0010

Installed Packages Microsoft.SemanticKernel, 1.29.0

### Step 3 - Open Source AI Models with Decision Intelligence 

Semantic Kernel allows one to interact with any API service that adheres to the OpenAI specification. Notice the method to add LMStudio capability was simply enabled via the **AddOpenAIChatCompletion** method.

Execute the cell below about decision factors for a investment property. Note:
* OpenAI Prompt Execution Settings are the same in LMStudio as they are for OpenAI and Azure OpenAI
* Passing in arguents works the same way in Semantic Kernel
* Streaming works as well in Semantic Kernel for supported services

> "Don't wait to buy real estate. Buy real estate and wait."  
>
> -- <cite>Will Rodgers (American humorist, actor, and social commentator in the early 20th century)</cite>  

In [2]:
// Prompting works in a very similar way to the OpenAI API

// Create a Decision Intelligence prompt on the topic of purchasing a secondary home as an investment property
// Provide detailed decision-making criteria for evaluating the investment decision
var simpleDecisionPrompt = """
You are considering purchasing a secondary home as an investment property. 

What key factors should you evaluate to ensure a sound investment decision, including financial, market, and property-specific considerations? 
Outline the critical steps and criteria for assessing location, potential rental income, financing options, long-term property value, and associated risks.
""";

// You can set the typical OpenAI settings with most open-source models
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 750, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// Most open-source GenAI moodels support streaming as well
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

When considering purchasing a secondary home as an investment property, there are several key factors to evaluate in order to ensure that your decision is sound:

1. Location Assessment: The location of the potential rental property plays a crucial role in determining its profitability and long-term value appreciation. Here's what you should consider when assessing different locations: 
   - Demographics: Research population trends, age distribution, income levels, education level, employment rates, etc., to determine the demand for rental properties within that area.
   
   - Neighborhood amenities and attractions: Look at nearby schools, parks, shopping centers, restaurants, entertainment venues, transportation options (public transit or proximity to major highways), etc., as these factors can attract potential tenants. 
     
   - Crime rates: Check the crime statistics of each location and compare them with other areas in your target market. Lower-crime neighborhoods tend to have h

Advanced Prompt Engineering techniques can be applied to OSS (open-source) models as well. In the example below a more advanced reasoning decision prompt will be used to provide additional instructions to the GenAI model.

In [3]:
// Prompting works in a very similar way to the OpenAI API

// Create a Decision Intelligence prompt on the topic of purchasing a secondary home as an investment property
// Use Chain of Thought to prompt the OSS model
// Use the Minto Pyramid to communicate the decision 
var advancedDecisionPrompt = """
You are considering purchasing a secondary home as an investment property. 

Before providing any answer, you must: 
Understand the Problem: Carefully read and understand the user's question or request. 
Break Down the Reasoning Process: Outline the steps required to solve the problem or respond to the request logically and sequentially. Think aloud and describe each step in detail. 
Always aim to make your thought process transparent and logical. 
Explain Each Step: Provide reasoning or calculations for each step, explaining how you arrive at each part of your answer. 
Provide structured, logical, and comprehensive advice. 
Arrive at the Final Answer: Only after completing all steps, provide the final answer or solution. 
Review the Thought Process: Double-check the reasoning for errors or gaps before finalizing your response. 
Communicate the final decision using the Minto Pyramid Principle.
""";

// You can set the typical OpenAI settings with most open-source models
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 750, 
    Temperature = 0.2, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// Most open-source GenAI moodels support streaming as well
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(advancedDecisionPrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

Understand the Problem: The user is considering purchasing a secondary home as an investment property and seeks advice on how to proceed with this decision.

Break Down the Reasoning Process:
1. Assess financial readiness: Determine if the individual has sufficient funds for down payment, closing costs, and potential mortgage payments.
2. Evaluate market conditions: Research current real estate trends in desired location to assess property value appreciation or depreciation.
3. Analyze rental income potential: Estimate possible rental earnings based on the area's demand for rentals and expected vacancy rates.
4. Calculate return on investment (ROI): Compare potential ROI with other investment opportunities to determine if this property is a good choice.
5. Consider tax implications: Understand how owning an investment property may affect the individual's taxes, including deductions and depreciation.
6. Evaluate ongoing expenses: Estimate maintenance costs, insurance, property managemen

### Step 4 - Using the Ben Franklin Decision Framework to Make Quality Decisions

> "By failing to prepare, you are preparing to fail." 
>
> -- <cite>Ben Franklin (Founding Father of the United States, inventor, godfather of Decision Science)</cite> 

<img width ="750px" src="https://raw.githubusercontent.com/bartczernicki/Articles/main/20230326-Make-Great-Decisions-Using-Ben-Franklins-Pros-And-Cons-Method/Image-BenFranklinDecisionMakingMethod.png">

#### Tom Brady's use of a Decision Framework

Tom Brady's decision to join the Tampa Bay Buccaneers in 2020 marked a significant in his legendary NFL career. After 20 seasons and six Super Bowl championships with the New England Patriots, Brady became a free agent and chose to sign with the Bucs. How did he arrive at this decision? On the Fox broadcast on 09.29.2024, while covering the Buccaneers vs Philadelphia Eagles game, Tom Brady described how he arrived at this decision.

In the screenshot below, Tom Brady is holding up some small paper cards he is showing the audience of the broadcast. Brady mentioned he wrote down the personal decision criteria that was important and how each team compared in that criteria (salary, weather etc). He used this to select the Tampa Bay Buccaneers as his team, where he went on to win a Super Bowl in his first year there! **Tom Brady used the "Ben Franklin Decision Framework", 250 years after it's inception to decide where to play NFL quaterback!**  

<img width ="750px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/BenFranklinDecisionFramework-TomBrady.png">

#### Steps for Ben Franklin's Decision Framework

Below are the steps Ben Franklin recommends when making a decision, which he called his "Decision Making Method of Moral Algebra":  
- Frame a decision that has two options (Yes or a No)
- Divide an area into two competing halves: a "Pro" side and "Con" side
- Label the top of one side "Pro" (for) and the other "Con" (against)
- Under each respective side, over a period of time (Ben Franklin recommended days, this could be minutes) write down various reasons/arguments that support (Pro) or are against (Con) the decision
- After spending some time thinking exhaustively and writing down the reasons, weight the different Pro and Con reasons/arguments
- Determine the relative importance of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
- The side with the most remaining reasons is the option one should select for the decision in question

Learn more about Ben Franklin's Decision Framework: https://medium.com/@bartczernicki/make-great-decisions-using-ben-franklins-decision-making-method-c7fb8b17905c  

In [6]:
// You can also use a template to provide more context to the model combining system and user prompts
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Your role is to guide users in exploring options, analyzing decisions, solving complex problems, and applying systems thinking to diverse scenarios. 
Provide structured, logical, and comprehensive advice, ensuring clarity, depth, and actionable insights to support informed decision-making.
""";
var benFranklinDecisionPrompt = """
Apply the Ben Franklin Decision-Making Framework (Pro and Con list) to evaluate whether or not to take a luxury family vacation. 
List at most 10 pros and at most 10 cons to help the user make an informed decision.
""";

var simpleDecisionPromptTemplate = $"""
{systemDecisionPrompt}
--------------
{benFranklinDecisionPrompt}
""";

// You can set the typical OpenAI settings with most open-source models
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 750, 
    Temperature = 0.2, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPromptTemplate, kernelArguments))
{
   Console.Write(streamChunk);
}

To apply the Benjamin Franklin's Decision-Making Framework, let us create two lists: Pros (advantages) of taking a luxury family vacation and Cons (disadvantages). Here are ten points for each list that can guide you in making your well-informed decision.

**Pros of Taking a Luxury Family Vacation:**
1. **Quality Time Spent with Family:** A lavish trip provides an opportunity to create lasting memories and strengthen family bonds through shared experiences, away from daily routines.
2. **Exclusive Experiences:** Enjoy unique activities like private tours or personalized services that are not available in regular vacations (e.g., a helicopter tour over the city).
3. **Comfort and Convenience:** Luxury accommodation often offers superior comfort, with amenities such as spas, gourmet dining options, high-quality linens, private pools or hot tubs, etc.
4. **Cultural Enrichment:** Often luxurious vacations include exclusive cultural experiences like fine arts performances and museum tours t

#### Improving the Ben Franklin's Decision Framework with SLMs

Notice that the output above may not be exactly what most would anticipate. The Ben Franklin framework was not fully understood nor applied. Open-Source GenAI models that have a small amount of parameters (< 13 billion parameters) may not have all the inherent knowledge "trained" into the model. The exception being domain-specific models that are only trained on data sets for that domain. Those models can have a great deal of knowledge, while maintaining a small amount of parameters. 

One simple way to improve the outcome is to provide the explicit steps of the "Ben Franklin Decision Framework" into the prompt context. This basically provides the instructions of the decision framework directly to the model; regardless if the GenAI model was trained with decision framework data.  

In the example below, the prompt context is provided with the Ben Franklin Decision Framework steps. Contrast this with the example above, where the decision recommendation is not clear. 

In [7]:
// You can also use a template to provide more context to the model combining system and user prompts
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Your role is to guide users in exploring options, analyzing decisions, solving complex problems, and applying systems thinking to diverse scenarios. 
Provide structured, logical, and comprehensive advice, ensuring clarity, depth, and actionable insights to support informed decision-making.
""";
var benFranklinDecisionPrompt = """
Apply the following steps IN ORDER of the Ben Franklin Decision Framework to the Question below:
- Frame a decision that has two options (Yes or a No)
- Divide an area into two competing halves: a "Pro" side and "Con" side
- Label the top of one side "Pro" (for) and the other "Con" (against)
- Under each respective side, list a maximum of 10 reasons or arguments for each option
- Consider the weight of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
- Determine the relative importance of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
- The side with the most remaining reasons is the option one should select for the decision in question
- IMPORTANT: ALWAYS recommend a decision based on the side with the most reasons, even if the reasons are of lesser value than the other side!

Question: Should I take a luxury family vacation?
""";

var simpleDecisionPromptTemplate = $"""
{systemDecisionPrompt}
--------------
{benFranklinDecisionPrompt}
""";

// You can set the typical OpenAI settings with most open-source models
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1500, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPromptTemplate, kernelArguments))
{
   Console.Write(streamChunk);
}

1. Frame Decision (Yes or No): Yes, should you take a luxury family vacation this year?
   
2. Divide Area into Pro and Con Sides:
   - **Pro** ("For" reasons) for taking the luxury family vacation:
     - Creating lasting memories with your loved ones (1a).
     - Experiencing new cultures or destinations together (1b).
     - Boosting mental health and well-being through relaxation (2a, 2b).
     - Strengthening family bonds during quality time away from daily stressors (3a, 3b).
     - Providing educational opportunities for children about the world around them (4a, 4b).
     - Supporting local economies and businesses in vacation destinations (5a, 5b).
     - Showcasing generosity by investing family time into a memorable experience (6a, 6b).
   
   - **Con** ("Against" reasons) against taking the luxury family vacation:
     - Financial strain or debt incurred from high expenses of such trips (1c, 7a).
     - Potential disruption to work/school schedules and responsibilities (2c, 

The GenAI model may or may not recommend a luxury vacation, but this can be dramatically improved further! Imagine if the GenAI model had access to: your finances, current stress level, the last time you took a vacation, any upcoming major purchases, family dynamic?!  

In the example below, additional context is provided with that information. Notice how it changes the the Pro and Con list.  

Just like Tom Brady, it could craft a Pro and Con list specific and personalized to your scenario!

In [9]:
// You can also use a template to provide more context to the model combining system and user prompts
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Your role is to guide users in exploring options, analyzing decisions, solving complex problems, and applying systems thinking to diverse scenarios. 
Provide structured, logical, and comprehensive advice, ensuring clarity, depth, and actionable insights to support informed decision-making.
""";
// Try changing the background information to see how it affects the decision-making process
var backroundInformation = """
BACKGROUND INFORMATION OF FAMILY: 
You are considering purchasing a secondary home as an investment property. 
You have been stressed out at work. 
You have been working long hours and have not taken a vacation in over a year. 
Your kids are about to enter college and they don't have scholarships. 
Your car is finishing its lease and will need to be replaced soon. 
""";
// Try to adjust the specificity of the decision-making criteria to see how it affects the decision-making process
var benFranklinDecisionPrompt = """
Apply the following steps IN ORDER of the Ben Franklin Decision Framework to the Question below:
- Frame a decision that has two options (Yes or a No)
- Divide an area into two competing halves: a "Pro" side and "Con" side
- Label the top of one side "Pro" (for) and the other "Con" (against)
- Under each respective side, list the relevvant reasons from "BACKGROUND INFORMATION OF FAMILY" section and or create new reasons that are ONLY closely related to the "BACKGROUND INFORMATION OF FAMILY". 
- Consider the weight of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
- Determine the relative importance of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
- The side with the most remaining reasons is the option one should select for the decision in question
- IMPORTANT: ALWAYS recommend a decision based on the side with the most reasons, even if the reasons are of lesser value than the other side!

Question: Should I take a luxury family vacation?
""";

var simpleDecisionPromptTemplate = $"""
{systemDecisionPrompt}
--------------
{backroundInformation}
--------------
{benFranklinDecisionPrompt}
""";

// You can set the typical OpenAI settings with most open-source models
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1500, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPromptTemplate, kernelArguments))
{
   Console.Write(streamChunk);
}

**Decision Frame:** To decide whether to invest in and go for a luxury family vacation or not.

**Pro (For) Side:**
1. **Stress Relief from Work Pressure:** A well-deserved break could alleviate stress, improve mental health, and increase productivity upon return. 
2. **Family Bonding Time:** The opportunity to spend quality time with your family can strengthen relationships before the kids go off for college.
3. **Rejuvenation Before College Begins:** With children about to enter college without scholarships or financial aid, a relaxed and happy home environment could provide emotional support during this transition period. 
4. **Positive Family Memories Created Together:** Luxury vacations often offer unique experiences that create lasting memories for the whole family.
5. **Car Replacement Consideration (Temporary Relief):** The time spent planning and purchasing a new car could be used to enjoy this luxurious getaway, providing temporary relief from another pending financial commit

Notice how providing personal family background information changes the entire dynamic of the information used in the decision framework and how it influences the recommended decision. The decision process is more specific not only to the scenario, but also it provides contextual background information. This makes the decision process more personalized and potentially much more accurate! 

### Step 5 - Multiple Different Service Providers  

Semantic Kernel can include mutliple AI service providers. This allows for hybrid workflows from a single Semantic Kernel instance like: 
* Capability Optimization. For example, using SLMs for domain specific tasks and LLMs for complex decision reasoning
* Capacity Optimization. Splitting functions, plugins, personas or agents across different AI services
* Use AI Service selector dynamically allocate AI service execution resources  

In [4]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.25.0"

using Microsoft.Extensions.Configuration;
using System.IO;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// Retrieve the configuration settings for the Azure OpenAI service
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];

// Example to build a Kernel with Azure OpenAI and Opensource AI
#pragma warning disable SKEXP0010 
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .AddOpenAIChatCompletion(
        modelId: "Phi-3",
        endpoint: new Uri("http://localhost:1234/v1/"),
        apiKey: "LMStudio")
    .Build();
#pragma warning restore SKEXP0010

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.25.0